In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName('Final_cleaning') \
.config('spark.shuffle.useOldFetchProtocol' , 'true') \
.config('spark.sql.warehouse.dir' , 'user/itv006879/warehouse') \
.enableHiveSupport() \
.master('yarn') \
.getOrCreate()

In [3]:
customers_df = spark.sql('select * from lending_club_proj_6879.customers')

In [ ]:
loans_defaulters_delinq_df = spark.sql('select * from lending_club_proj_6879.loans_defaulters_delinq')

In [ ]:
loans_defaulters_detail_rec_enq_df = spark.sql('select * from lending_club_proj_6879.loans_defaulters_detail_rec_enq')

In [4]:
from pyspark.sql.functions import *
bad_customer_ids_1 = customers_df.groupBy('member_id').agg(count('*').alias('total')).orderBy(col('total').desc()).where(col('total') > 1).select('member_id')

In [5]:
bad_customer_ids_2 = loans_defaulters_delinq_df.groupBy('member_id').agg(count('*').alias('total')).orderBy(col('total').desc()).where(col('total') > 1).select('member_id')

In [6]:
bad_customer_ids_3 = loans_defaulters_detail_rec_enq_df.groupBy('member_id').agg(count('*').alias('total')).orderBy(col('total').desc()).where(col('total') > 1).select('member_id')

In [9]:
bad_data_customers = bad_customer_ids_1.union(bad_customer_ids_2).union(bad_customer_ids_3).distinct()

In [18]:
bad_data_customers.repartition(1).write \
.format('csv') \
.mode('overwrite') \
.option('header' , 'true') \
.option('path' , 'user/itv006879/bad_data/') \
.save()

In [10]:
bad_customers_data.createOrReplaceTempView('bad_data')

In [12]:
customer_df = customers_df.where(" member_id not in (select * from bad_data) ")

In [13]:
loans_defaulters_delinq_df = loans_defaulters_delinq_df.where(" member_id not in (select * from bad_data) ")

In [14]:
loans_defaulters_detail_rec_enq_df = loans_defaulters_detail_rec_enq_df.where(" member_id not in (select * from bad_data) ")

In [19]:
customer_df.write \
.mode('overwrite') \
.option('path' , '/user/itv006879/refined_data/customer') \
.save()

In [21]:
loans_defaulters_delinq_df.write \
.mode('overwrite') \
.option('header' , 'true') \
.option('path' , '/user/itv006879/refined_data/loans_defaulters_delinq') \
.save()

In [22]:
loans_defaulters_detail_rec_enq_df.write \
.mode('overwrite') \
.option('header' , 'true') \
.option('path' , '/user/itv006879/refined_data/loans_defaulters_detail_rec_enq') \
.save()

In [ ]:
spark.sql(''' 
create EXTERNAL TABLE lending_club_proj_6879.customers(member_id string,emp_title
string,emp_length int, home_ownership string,annual_income
float,address_state string,address_zipcode string,address_country
string,grade string,sub_grade string,verification_status
string,total_high_credit_limit float,application_type
string,join_annual_income float,verification_status_joint string,ingest_date
timestamp) stored as parquet location '/public/trendytech/lendingclubproject/re-cleaned/customers_new'
''')

In [ ]:
spark.sql("""
CREATE EXTERNAL TABLE lending_club_proj_6879.loans_defaulters_delinq(member_id
string,delinq_2yrs integer,delinq_amnt float,mths_since_last_delinq integer)
stored as parquet LOCATION '/public/trendytech/lendingclubproject/re-cleaned/loans_defaulters_delinq_new'
""")

In [ ]:
spark.sql("""
CREATE EXTERNAL TABLE lending_club_proj_6879.loans_defaulters_detail_rec_enq(member_id string,
pub_rec integer, pub_rec_bankruptcies integer, inq_last_6mths integer) stored
as parquet LOCATION '/public/trendytech/lendingclubproject/re-cleaned/loans_defaulters_detail_records_enq_new'
""")
